In [8]:
import os
import sqlite3
import requests
import pandas as pd

In [10]:
RAPIDAPI_HOST = os.environ.get("RAPIDAPI_HOST")
RAPIDAPI_KEY = os.environ.get("RAPIDAPI_KEY")

In [ ]:
local_path = os.getcwd()
local_path

In [15]:
HEADERS = {
    "x-rapidapi-key": RAPIDAPI_KEY,
    "x-rapidapi-host": RAPIDAPI_HOST
}
BASE_URL = f"https://{RAPIDAPI_HOST}/"

In [16]:
def get(endpoint, params=None):
    r = requests.get(f"{BASE_URL}/{endpoint}", headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

In [ ]:
def extract_teams():
    data = get("nfl-team-listing/v1/data")
    extracted = []
    for item in data:
        team = item.get("team", {})
        extracted.append({
            "team_id": team.get("id"),
            "display_name": team.get("displayName"),
            "name": team.get("name"),
            "nickname": team.get("nickname"),
            "location": team.get("location")
        })
    return pd.DataFrame(extracted)

In [28]:
df = extract_teams()
df

,team_id,display_name,name,nickname,location
0,22,Arizona Cardinals,Cardinals,Cardinals,Arizona
1,1,Atlanta Falcons,Falcons,Falcons,Atlanta
2,33,Baltimore Ravens,Ravens,Ravens,Baltimore
3,2,Buffalo Bills,Bills,Bills,Buffalo
4,29,Carolina Panthers,Panthers,Panthers,Carolina
5,3,Chicago Bears,Bears,Bears,Chicago
6,4,Cincinnati Bengals,Bengals,Bengals,Cincinnati
7,5,Cleveland Browns,Browns,Browns,Cleveland
8,6,Dallas Cowboys,Cowboys,Cowboys,Dallas
9,7,Denver Broncos,Broncos,Broncos,Denver
